# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff82d7488b0>
├── 'a' --> tensor([[ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077]])
└── 'x' --> <FastTreeValue 0x7ff82d748ca0>
    └── 'c' --> tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                        [-2.1943, -0.1718, -0.8100, -0.2232],
                        [-1.2083, -0.7442,  0.0732, -0.5177]])

In [4]:
t.a

tensor([[ 0.7115, -1.1736,  0.1105],
        [-0.0873, -1.5734,  0.1077]])

In [5]:
%timeit t.a

65.9 ns ± 0.0164 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff82d7488b0>
├── 'a' --> tensor([[ 0.0409, -0.3651,  0.2265],
│                   [ 0.3102,  0.1575,  0.3460]])
└── 'x' --> <FastTreeValue 0x7ff82d748ca0>
    └── 'c' --> tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                        [-2.1943, -0.1718, -0.8100, -0.2232],
                        [-1.2083, -0.7442,  0.0732, -0.5177]])

In [7]:
%timeit t.a = new_value

73.9 ns ± 0.0118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7115, -1.1736,  0.1105],
               [-0.0873, -1.5734,  0.1077]]),
    x: Batch(
           c: tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7115, -1.1736,  0.1105],
        [-0.0873, -1.5734,  0.1077]])

In [11]:
%timeit b.a

66.9 ns ± 0.0482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0795,  1.7870, -0.1324],
               [-0.6320, -0.2202,  0.2611]]),
    x: Batch(
           c: tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.249 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 579 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff82d748a60>
├── 'a' --> tensor([[[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]],
│           
│                   [[ 0.7115, -1.1736,  0.1105],
│                    [-0.0873, -1.5734,  0.1077]]])
└── 'x' --> <FastTreeValue 0x7ff784f90ee0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 72.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff77a980340>
├── 'a' --> tensor([[ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077],
│                   [ 0.7115, -1.1736,  0.1105],
│                   [-0.0873, -1.5734,  0.1077]])
└── 'x' --> <FastTreeValue 0x7ff828660130>
    └── 'c' --> tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                        [-2.1943, -0.1718, -0.8100, -0.2232],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.7 µs ± 57.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0508,  0.6248, -1.1841,  0.9425],
                       [-2.1943, -0.1718, -0.8100, -0.2232],
                       [-1.2083, -0.7442,  0.0732, -0.5177]],
              
                      [[ 1.0508,  0.6248, -1.1841,  0.9425],
                       [-2.1943, -0.1718, -0.8100, -0.2232],
                       [-1.2083, -0.7442,  0.0732, -0.5177]],
              
                      [[ 1.0508,  0.6248, -1.1841,  0.9425],
                       [-2.1943, -0.1718, -0.8100, -0.2232],
                       [-1.2083, -0.7442,  0.0732, -0.5177]],
              
                      [[ 1.0508,  0.6248, -1.1841,  0.9425],
                       [-2.1943, -0.1718, -0.8100, -0.2232],
                       [-1.2083, -0.7442,  0.0732, -0.5177]],
              
                      [[ 1.0508,  0.6248, -1.1841,  0.9425],
                       [-2.1943, -0.1718, -0.8100, -0.2232],
                       [-1.2083, -0.7442,  0.0732, -0.5177]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177],
                      [ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177],
                      [ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177],
                      [ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177],
                      [ 1.0508,  0.6248, -1.1841,  0.9425],
                      [-2.1943, -0.1718, -0.8100, -0.2232],
                      [-1.2083, -0.7442,  0.0732, -0.5177],
                      [ 1.0508,  0.6248, -1.1841,  0.9425],
                   

In [28]:
%timeit Batch.cat(batches)

140 µs ± 430 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 3.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
